In [ ]:
!nvidia-smi

Fri Oct  9 15:51:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 8.1MB/s 
     |████████████████████████████████| 1.1MB 33.2MB/s 
     |████████████████████████████████| 317kB 38.9MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 7.4MB 25.2MB/s 
     |████████████████████████████████| 1.7MB 61.0MB/s 
     |████████████████████████████████| 1.1MB 54.4MB/s 
     |████████████████████████████████| 890kB 30.8MB/s 
     |████████████████████████████████| 133kB 55.2MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 112kB 59.8MB/s 
     |████████████████████████████████| 4.4MB 56.3MB/s 
     |████████████████████████████████| 6.7MB 55.7MB/s 
     |████████████████████████████████| 122kB 56.5MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 163kB 58.2MB/s 
     |████████████████████████████████| 71kB 10.6MB/

In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

In [2]:
users = pd.read_csv('users_interactions.csv')
users = users.loc[:,['contentId', 'eventType']]

articles = pd.read_csv('shared_articles.csv')

# filtering for english language content and created content only
articles = articles[(articles['eventType'] == 'CONTENT SHARED') & (articles.lang == 'en')]
articles = articles.loc[:,['contentId', 'title', 'text', 'timestamp']]

In [3]:
users['count'] = 1      
users = users.groupby(['contentId','eventType'])['count'].sum().reset_index()

users = users.pivot_table('count', ['contentId'], 'eventType')

users = users.fillna(0)

users['VIRALITY'] = (1*users['VIEW']) + (4*users['LIKE']) + (10*users['COMMENT CREATED']) +(25*users['FOLLOW'] )+ (100*users['BOOKMARK'])

users.contentId = users.index

data = pd.merge(users, articles, on='contentId', how='inner')

C:\Users\jashj\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.


In [4]:
def time_func(hour):
  if 0<=hour<6:
    return 'night'

  elif 6<=hour<12:
    return 'morning'

  elif 12<=hour<18:
    return 'afternoon'

  elif 18<=hour<24:
    return 'evening'

In [5]:
data['timestamp'] = pd.to_datetime(data['timestamp'], unit = 's')
data['timestamp'] = data['timestamp'].dt.hour
data['timestamp'] = data['timestamp'].apply(time_func)

In [6]:
data.head()

,contentId,BOOKMARK,COMMENT CREATED,FOLLOW,LIKE,VIEW,VIRALITY,title,text,timestamp
0,-9192549002213406534,0.0,1.0,0.0,5.0,50.0,80.0,Chrome OS now has Material Design for the desktop,Chromebook users are in for a surprise: Chrome...,evening
1,-9190737901804729417,0.0,0.0,0.0,1.0,8.0,12.0,The Real Product Market Fit,"Michael Seibel, Y Combinator Partner, on knowi...",afternoon
2,-9189659052158407108,2.0,0.0,0.0,1.0,33.0,237.0,Eddy Cue and Craig Federighi Open Up About Lea...,Fast Company : Let's start talking about Maps....,night
3,-9171475473795142532,1.0,0.0,0.0,0.0,4.0,104.0,Decentralized Options Exchange Etheropt Uses A...,The price per Ether will be taken from multipl...,morning
4,-9166778629773133902,0.0,0.0,0.0,0.0,5.0,5.0,Google CDN Beta is here... and it's already on...,"Some months ago, Google launched their Alpha p...",afternoon


In [18]:
df=pd.DataFrame()

# df['text']=data['title']+data['text']
# df['labels']=data['VIEW']

In [19]:
df['text_a']=data['title']
df['text_b']=data['text']
df['labels']=data['LIKE']

In [20]:
# a,b=data.iloc[1][['title','text']].values

In [21]:
df.reset_index(drop=True, inplace=True)

In [22]:
df

,text_a,text_b,labels
0,Chrome OS now has Material Design for the desktop,Chromebook users are in for a surprise: Chrome...,5.0
1,The Real Product Market Fit,"Michael Seibel, Y Combinator Partner, on knowi...",1.0
2,Eddy Cue and Craig Federighi Open Up About Lea...,Fast Company : Let's start talking about Maps....,1.0
3,Decentralized Options Exchange Etheropt Uses A...,The price per Ether will be taken from multipl...,0.0
4,Google CDN Beta is here... and it's already on...,"Some months ago, Google launched their Alpha p...",0.0
...,...,...,...
2144,Google search now provides access to basic hea...,Google's latest move in India may help increas...,0.0
2145,"Meet Mycroft, the open source AI who wants to ...",Image: Mycroft It might bear a passing resembl...,3.0
2146,Ray Kurzweil Predicts Three Technologies Will ...,"55 0 Over the last several decades, the digita...",0.0
2147,Angular 2,Welcome to the Angular 2 Style Guide Purpose I...,2.0


In [23]:
bert_args = {
    "reprocess_input_data": False,
    "overwrite_output_dir": True,
    "num_train_epochs": 4,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "learning_rate": 5e-4,
    "warmup_steps": 10000,
    "train_batch_size": 16,
    "eval_batch_size": 2,
    "gradient_accumulation_steps": 1,
    "block_size": 128,
    "max_seq_length": 512,
    "dataset_type": "simple",
#     "wandb_project": "Esperanto - ELECTRA",
#     "wandb_kwargs": {"name": "Electra-SMALL"},
    "logging_steps": 100,
    "evaluate_during_training": False,
    "evaluate_during_training_steps": 50000,
    "evaluate_during_training_verbose": False,
    "use_cached_eval_features": True,
    "sliding_window": False,
    'regression': True,
}

In [24]:
electra_args = {
    "reprocess_input_data": False,
    "overwrite_output_dir": True,
    "num_train_epochs": 1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "learning_rate": 5e-4,
    "warmup_steps": 10000,
    "train_batch_size": 16,
    "eval_batch_size": 2,
    "gradient_accumulation_steps": 1,
    "block_size": 128,
    "max_seq_length": 512,
    "dataset_type": "simple",
#     "wandb_project": "Esperanto - ELECTRA",
#     "wandb_kwargs": {"name": "Electra-SMALL"},
    "logging_steps": 100,
    "evaluate_during_training": False,
    "evaluate_during_training_steps": 50000,
    "evaluate_during_training_verbose": False,
    "use_cached_eval_features": True,
    "sliding_window": False,
    'regression': True,
#     "vocab_size": 52000,
    "generator_config": {
        "embedding_size": 128,
        "hidden_size": 256,
        "num_hidden_layers": 3,
    },
    "discriminator_config": {
        "embedding_size": 128,
        "hidden_size": 256,
    },
}

In [27]:
model = ClassificationModel('bert', 'bert-base-cased', num_labels=1, use_cuda=True, cuda_device=0, args=bert_args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [28]:
model.train_model(train_df=df)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


C:\Users\jashj\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


RuntimeError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 6.00 GiB total capacity; 4.47 GiB already allocated; 187.62 MiB free; 4.51 GiB reserved in total by PyTorch)

In [ ]:
# result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [128]:
# predictions, raw_outputs = model.predict([["I'd like to puts some CD-ROMS on my iPad, is that possible?'", "Yes, but wouldn't that block the screen?"]])
# print(predictions)
# print(raw_outputs)

In [129]:
df.iloc[0]['text']

'Chrome OS now has Material Design for the desktopChromebook users are in for a surprise: Chrome 50 is bringing material design to Chrome OS, too. On his blog , Senior Designer at Google Sebastien Gabriel notes there will also be two implementations of Chrome OS. One will be designed for those using mice, while touch-enabled Chromebooks will have a "hybrid" layout employing more space between buttons and icons. Material Design for Chrome OS is a lot like you\'ll find for Android , and that\'s probably the point. As we\'ve detailed before , Google wants Material Design to be the design language more developers and designers turn to as a default, especially with its recursive look and feel. It also makes sense for Chrome OS, which more heavily on the Web than more traditional operating systems. If Chrome as a browser is going to sport Material Design, its OS should too. Read next: Get ready to kill your cable box: Xfinity is coming to Roku and Samsung Smart TVs TNW\'s West Coast writer i

In [134]:
a,b=data.iloc[0][['title','text']].values
predictions, raw_outputs = model.predict([[a, b]])
print(predictions)
print(raw_outputs)


0.27490234
0.27490234


MULTILABEL

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
df['text_a']=data['title']
df['text_b']=data['text']
df['labels']=data['VIRALITY']

In [ ]:
data['labels']=list(zip(data.BOOKMARK.tolist(), data.FOLLOW.tolist(), data.LIKE.tolist(), data.VIEW.tolist(),  data.VIRALITY.tolist()))
data.head()

,contentId,BOOKMARK,COMMENT CREATED,FOLLOW,LIKE,VIEW,VIRALITY,title,text,timestamp,labels
0,-9192549002213406534,0.0,1.0,0.0,5.0,50.0,80.0,Chrome OS now has Material Design for the desktop,Chromebook users are in for a surprise: Chrome...,evening,"(0.0, 0.0, 5.0, 50.0, 80.0)"
1,-9190737901804729417,0.0,0.0,0.0,1.0,8.0,12.0,The Real Product Market Fit,"Michael Seibel, Y Combinator Partner, on knowi...",afternoon,"(0.0, 0.0, 1.0, 8.0, 12.0)"
2,-9189659052158407108,2.0,0.0,0.0,1.0,33.0,237.0,Eddy Cue and Craig Federighi Open Up About Lea...,Fast Company : Let's start talking about Maps....,night,"(2.0, 0.0, 1.0, 33.0, 237.0)"
3,-9171475473795142532,1.0,0.0,0.0,0.0,4.0,104.0,Decentralized Options Exchange Etheropt Uses A...,The price per Ether will be taken from multipl...,morning,"(1.0, 0.0, 0.0, 4.0, 104.0)"
4,-9166778629773133902,0.0,0.0,0.0,0.0,5.0,5.0,Google CDN Beta is here... and it's already on...,"Some months ago, Google launched their Alpha p...",afternoon,"(0.0, 0.0, 0.0, 5.0, 5.0)"


In [ ]:
data_final=data[['title','text','labels']]
data_final.head()

,title,text,labels
0,Chrome OS now has Material Design for the desktop,Chromebook users are in for a surprise: Chrome...,"(0.0, 0.0, 5.0, 50.0, 80.0)"
1,The Real Product Market Fit,"Michael Seibel, Y Combinator Partner, on knowi...","(0.0, 0.0, 1.0, 8.0, 12.0)"
2,Eddy Cue and Craig Federighi Open Up About Lea...,Fast Company : Let's start talking about Maps....,"(2.0, 0.0, 1.0, 33.0, 237.0)"
3,Decentralized Options Exchange Etheropt Uses A...,The price per Ether will be taken from multipl...,"(1.0, 0.0, 0.0, 4.0, 104.0)"
4,Google CDN Beta is here... and it's already on...,"Some months ago, Google launched their Alpha p...","(0.0, 0.0, 0.0, 5.0, 5.0)"


In [ ]:
args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": False,
    "fp16_opt_level": "O0",
    "max_seq_length": 128,
    "train_batch_size": 64,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 2,
    "weight_decay": 0,
    "learning_rate": 5e-4,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 100,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": True,
    "evaluate_during_training": False,
    'regression': True,

#     "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
}

In [ ]:
# model = MultiLabelClassificationModel('bert', 'bert-base-cased', num_labels=11, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 5})
model = MultiLabelClassificationModel('bert', 'bert-base-cased', num_labels=5, args=args)

INFO:filelock:Lock 139801388395768 acquired on /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock


INFO:filelock:Lock 139801388395768 released on /root/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock
INFO:filelock:Lock 139801388393472 acquired on /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2.lock


INFO:filelock:Lock 139801388393472 released on /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2.lock


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not in

INFO:filelock:Lock 139801388396384 released on /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


In [ ]:
model.train_model(data_final)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


In [ ]:
predictions, raw_outputs = model.predict([["I'd like to puts some CD-ROMS on my iPad, is that possible?'", "Yes, but wouldn't that block the screen?"]])
print(predictions)
print(raw_outputs)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



[[1, 0, 1, 1, 1]]
[[0.84968245 0.38917312 0.99960667 0.99995911 1.        ]]


In [ ]:
predictions, raw_outputs = model.predict([["Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's", "All of this work is still very early. The first full public version of the Ethereum software was recently released, and the system could face some of the same technical and legal problems that have tarnished Bitcoin. Many Bitcoin advocates say Ethereum will face more security problems than Bitcoin because of the greater complexity of the software. Thus far, Ethereum has faced much less testing, and many fewer attacks, than Bitcoin. The novel design of Ethereum may also invite intense scrutiny by authorities given that potentially fraudulent contracts, like the Ponzi schemes, can be written directly into the Ethereum system. But the sophisticated capabilities of the system have made it fascinating to some executives in corporate America. IBM said last year that it was experimenting with Ethereum as a way to control real world objects in the so-called Internet of things. Microsoft has been working on several projects that make it easier to use Ethereum on its computing cloud, Azure. 'Ethereum is a general platform where you can solve problems in many industries using a fairly elegant solution - the most elegant solution we have seen to date,' said Marley Gray, a director of business development and strategy at Microsoft. Mr. Gray is responsible for Microsoft's work with blockchains, the database concept that Bitcoin introduced. Blockchains are designed to store transactions and data without requiring any central authority or repository. Blockchain ledgers are generally maintained and updated by networks of computers working together - somewhat similar to the way that Wikipedia is updated and maintained by all its users. Many corporations, though, have created their own Ethereum networks with private blockchains, independent of the public system, and that could ultimately detract from the value of the individual unit in the Ethereum system - known as an Ether - that people have recently been buying. The interest in Ethereum is one sign of the corporate fascination with blockchains. Most major banks have expressed an interest in using them to make trading and money transfer faster and more efficient. On Tuesday, executives from the largest banks will gather for a conference, 'Blockchain: Tapping I nto the Real Potential , Cutting Through the Hype.' Many of these banks have recently been looking at how some version of Ethereum might be put to use. JPMorgan, for instance, has created a specific tool, Masala, that allows some of its internal databases to interact with an Ethereum blockchain. Michael Novogratz, a former top executive at the private equity firm Fortress Investing Group, who helped lead Fortress's investment in Bitcoin, has been looking at Ethereum since he left Fortress last fall. Mr. Novogratz said that he made a 'significant' purchase of Ether in January. He has also heard how the financial industry's chatter about the virtual currency has evolved. 'A lot of the more established players were thinking, 'It's still an experiment,' ' he said. 'It feels like in the last two to three months that experiment is at least getting a lot more validation.' Since the beginning of the year, the value of an individual unit of Ether has soared as high as $12 from around $1. That has brought the value of all existing Ether to over $1 billion at times, significantly more than any virtual currency other than Bitcoin, which had over $6 billion in value outstanding last week. Since Bitcoin was invented, there have been many so-called alt-coins that have tried to improve on Bitcoin, but none have won the following of Ethereum. Unlike Bitcoin, which was released in 2009 by a mysterious creator known as Satoshi Nakamoto, Ethereum was created in a more transparent fashion by a 21-year-old Russian-Canadian, Vitalik Buterin, after he dropped out of Waterloo University in Ontario. The most basic aim of Ethereum was to make it possible to program binding agreements into the blockchain - the smart contract concept. Two people, for instance, could program a bet on a sports game directly into the Ethereum blockchain. Once the final score came in from a mutually agreed upon source - say, The Associated Press - the money would be automatically transferred to the winning party. Ether can be used as a currency in this system, but Ether are also necessary to pay for the network power needed to process the bet. The Ethereum system has sometimes been described as a single shared computer that is run by the network of users and on which resources are parceled out and paid for by Ether. A team of seven co-founders helped Mr. Buterin write up the software after he released the initial description of the system. Mr. Buterin's team raised $18 million in 2014 through a presale of Ether, which helped fund the Ethereum Foundation, which supports the software's development. Like Bitcoin, Ethereum has succeeded by attracting a dedicated network of followers who have helped support the software, partly in the hope that their Ether will increase in value if the system succeeds. Last week, there were 5,800 computers - or nodes - helping support the network around the world. The Bitcoin network had about 7,400 nodes. One of Mr. Buterin's co-founders, Joseph Lubin, has set up ConsenSys, a company based in Brooklyn that has hired over 50 developers to build applications on the Ethereum system, including one that enables music distribution and another that allows for a new kind of financial auditing. The ConsenSys offices are in an old industrial building in the Bushwick section of Brooklyn. The office is essentially one large room, with all the messy trademarks of a start-up operation, including white boards on the walls and computer parts lying around. Mr. Lubin said he had thrown himself into Ethereum after starting to think that it delivered on some of the failed promise of Bitcoin, especially when it came to allowing new kinds of online contracts and markets. 'Bitcoin presented the broad strokes vision, and Ethereum presented the crystallization of how to deliver that vision,' he said. Joseph Bonneau, a computer science researcher at Stanford who studies so-called crypto-currencies, said Ethereum was the first system that had really caught his interest since Bitcoin. It is far from a sure thing, he cautioned. 'Bitcoin is still probably the safest bet, but Ethereum is certainly No. 2, and some folks will say it is more likely to be around in 10 years,' Mr. Bonneau said. 'It will depend if any real markets develop around it. If there is some actual application.'"]])
print(predictions)
print(raw_outputs)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



[[1, 0, 1, 1, 1]]
[[0.84968001 0.38917601 0.99960667 0.99995911 1.        ]]
